# Paneldata (time-series cross-section)

**Tvärsnittsdata** (cross-section) innebär att vi har data från många analysenheter, vid en tidpunkt.<br>
**Tidsseriedata** (time-series) innebär att vi har data från en analysenhet, vid många tidpunkter.<br>
**Paneldata** (time-series cross-section) innebär att vi har data från många analysenheter, vid många tidpunkter.

Med paneldata kan man göra intressantare analyser än med både tvärsnittsdata och tidsseriedata, som ger oss bättre möjligheter att utesluta alternativa förklaringar (och därmed göra det lättare att tala om orsak och verkan).

I Stata kan man använda tidsseriekommandon ([se separat guide för dem!](timesseries_sv.html)) i paneldata för att skapa laggade och ledande variabler. Vi kan också använda speciella regressionkommandon som är anpassade till paneldata, som `xtreg`.

**Men först måste man berätta för Stata att datan är paneldata. Om detta handlar denna guide.**

## Paneldatastrukturen: long eller wide

Paneldata kan vara strukturerad på två olika sätt: "long" eller "wide", alltså "lång" eller "bred". Säg till exempel att vi har data på länder, över tid.

### Bred (wide) data
Med bred (wide) data motsvara varje rad i datasetet ett land, och varje kolumn en variabel, vid en tidpunkt. Till exempel ett lands befolkningsstorlek, ett visst år. Såhär:

| country | population2000  | population2001  | population2002  |
|---------|-----------------|-----------------|-----------------|
| Sweden  | 8872284         | 8888675         | 8911899         |
| Norway  | 4491572         | 4514907         | 4537240         |

Det kan verka intuitivt vid första anblicken, och gör det lätt att jämföra enskilda år. Men det blir svårare att göra mer avancerade analyser, och med många olika variabler (befolkning, BNP, arbetslöshet, osv) blir det väldigt många kolumner.

### Lång (long) data
Oftast är det istället bättre att ha datan i långt format. Där är varje rad ett land ett år, och varje kolumn en variabel. Vi måste bara se till att ha en variabel som visar vilket år det är. Ovanstående tabell i långt format skulle se ut såhär:

| country | year     | population      |
|---------|----------|-----------------|
| Sweden  | 2000     | 8872284         | 
| Sweden  | 2001     | 8888675         | 
| Sweden  | 2002     | 8911899         | 
| Norway  | 2000     | 4491572         | 
| Norway  | 2001     | 4514907         | 
| Norway  | 2002     | 4537240         |

Samma data, i ett annat format. Här får vi istället en väldig massa rader, men det är lättare att hantera i Stata. För att byta format från wide till long, eller från long till wide, använd kommandot `reshape`. Om detta kommer en egen guide. Resten av guiden förutsätter att datan är i långt format. 

## Ange paneldatastrukturen med `xtset`
Vi behöver ange två variabler för Stata: en panelvariabel och en tidsvariabel. Panelvariabeln är i det här fallet land - alla observationer för Sverige hänger ihop, alla observationer för Norge hänger ihop, osv. Tidsvariabeln är i det här fallet år.

Vi gör det med kommandot `xtset`. Man skriver bara `xtset country year` till exempel. Låt oss prova, med QoG-institutets paneldatamängd, som innehåller just information om länder, över tid. Datan är i långt format.

In [8]:
use "https://www.qogdata.pol.gu.se/dataarchive/qog_bas_ts_jan18.dta", clear

(Quality of Government Basic dataset 2018 - Time-Series)


### Ett vanligt problem: panelvariabeln är en textsträng
I variabeln "cname" står det vad varje land heter, och i variabeln "year" står det vilket år det är. Men om vi prövar att skriva in detta med `xtset` så får vi ett felmeddelande:

In [9]:
xtset cname year

string variables not allowed in varlist;
cname is a string variable


r(109);


Stata klagar här på att panelvariabeln "cname" är text. Stata vill att det ska vara en sifferkod. I QoG-datan finns det sådana variabler, till exempel variabeln "ccode". Men i andra fall, till exempel när vi sätter ihop datan själv, kanske det inte finns någon sådan kod. Då får vi göra en själv. Det gör man lätt med kommandot `egen` kombinerat med `group()`:

In [17]:
egen countryid = group(cname)

Det som händer här är att Stata går igenom alla unika värden på variabeln "cname". Varje unikt värde får en egen sifferkod, från 1 och uppåt. Den här variabeln kan vi nu använda som vår panelvariabel.

### Ställa in paneldatan på rätt sätt

In [20]:
xtset countryid year

       panel variable:  countryid (strongly balanced)
        time variable:  year, 1946 to 2017
                delta:  1 unit


Nu fungerade det! Vi ser att vår nya variabel "countryid" är panelvariabeln, och att tidsvariabeln är year.

### Ett vanligt problem: Dubletter (repeated time values within panel)

Ett annat vanligt felmeddelande man kan få är "repeated time values within panel". Det betyder att det finns dubletter. De två variablerna man ställer in med `xtset` måste ge unika kombination för alla observationer. Om till exempel Sverige år 2000 finns med flera gånger i datasetet så vet Stata inte hur de observationerna ska hanteras, och då får man felmeddelandet.

Såhär ser det ut:

In [32]:
xtset countryid year

repeated time values within panel


r(451);


Tyvärr berättar Stata inte närmare vilka observationer det är som är problematiska. Men vi kan använda kommandot `duplicates` för att hitta dem. Vi skriver då `duplicates list` följt av variablerna i fråga, båda två. Om vi till exempel skriver `duplicates list countryid` så kommer vi få ut en massa observationer, eftersom varje land finns med många gånger. Men om vi skriver `duplicates list countryid year` får vi bara ut de observationer som är dubletter på båda variablerna.

In [36]:
duplicates list countryid year


Duplicates in terms of countryid year

  +-------------------------+
  |  obs:   countr~d   year |
  |-------------------------|
  | 15193        483   2000 |
  | 15194        483   2000 |
  | 15195        483   2000 |
  | 15196        483   2000 |
  | 15197        483   2000 |
  |-------------------------|
  | 15198        483   2000 |
  | 15199        483   2000 |
  | 15200        483   2000 |
  +-------------------------+


Här ser vi nu att det är 8 observationer som ställer till det. De har värdet 483 på variabeln countryid, och värdet 2000 på variabeln year.

När vi nu vet vilka observationer det är som strular behöver vi fundera på varför de finns med flera gånger. I det här fallet är det för att jag själv skapade dem, för att kunna visa felmeddelandet. Det går alltså bra att bara ta bort dem. Men i verkligheten är det inte givet vilka observationer som är de riktiga, så att säga. Då kan man inte bara ta bort dem hur som helst. Här gäller det alltså att ställa en diagnos just för det enskilda fallet.

Om man vill ta bort dem använder man i alla fall kommandot `drop` i kombination med en [if-sats](ifstatements_sv.html). Här nedan anger vi att alla observationer med värdet 483 på countryid och 2000 på year ska tas bort.

In [38]:
drop if countryid==483 & year==2000

(8 observations deleted)


Och efter det ska det gå bra att använda `xtset` som det är tänkt.

In [39]:
xtset countryid year

       panel variable:  countryid (strongly balanced)
        time variable:  year, 1946 to 2017
                delta:  1 unit


## Skapa laggade variabler
Nu är vi redo att börja använda datan. Vi kan till exempel prova att göra en laggad variabel som visar befolkningen föregående år. Här använder vi vanliga [tidsseriekommandon](timeseries_sv.html).

In [41]:
gen lag_pop = l.unna_pop

(7,121 missing values generated)


Om vi nu tittar på ett utsnitt av datan så ser vi att det fungerade:

In [42]:
list cname year unna_pop lag_pop if cname=="Sweden" & year>2010


       +------------------------------------+
       |  cname   year   unna_pop   lag_pop |
       |------------------------------------|
12738. | Sweden   2011    9462352   9382297 |
12739. | Sweden   2012    9543457   9462352 |
12740. | Sweden   2013    9624247   9543457 |
12741. | Sweden   2014    9703247   9624247 |
12742. | Sweden   2015    9779426   9703247 |
       |------------------------------------|
12743. | Sweden   2016          .   9779426 |
12744. | Sweden   2017          .         . |
       +------------------------------------+


Värdet på variabeln "unna_pop" är för Sverige år 2011 9462352 personer. Året efter har variabeln lag_pop också värdet 9462352. Som planerat! Det finurliga är också nu att Stata inte bara flyttat alla rader i kolumnen ett steg, utan tar hänsyn till att länderna byts. Det ena landets sista år blir inte till det första i nästa eller så.

## Avslutning
Det kan vara lite trix med att ställa in paneldata på rätt sätt, så att Stata fattar hur datan ska hanteras. Det är viktigt att datan är i långt format, så att varje rad är ett landår, och att man har en egen variabel som visar vilket år det är. Med kommandot `reshape` kan man omvandla dataset från långt till brett, eller från brett till långt. Om detta kommer det en egen guide.